In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')

In [14]:
### read the tab separated file containing healthcare provider data as a pandas DataFrame

provider_df = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2017_subset.txt', "\t")

provider_df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
0,1,CPT copyright 2016 American Medical Associatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99217,Hospital observation care discharge,N,100.0,96.0,100.0,73.398800,325.780000,56.827200,57.492400
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99218,Hospital observation care typically 30 minutes,N,26.0,25.0,26.0,100.080000,449.000000,78.460000,79.306154
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99219,Hospital observation care typically 50 minutes,N,52.0,51.0,52.0,136.380000,614.000000,102.807692,103.895385
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,99220,Hospital observation care typically 70 minutes...,N,59.0,59.0,59.0,190.363729,755.932203,141.293559,142.865763


In [15]:
convicted_df = pd.read_csv('LEIE.csv')
convicted_df.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN


In [16]:
provider_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 26 columns):
npi                                 99999 non-null int64
nppes_provider_last_org_name        99999 non-null object
nppes_provider_first_name           96084 non-null object
nppes_provider_mi                   71210 non-null object
nppes_credentials                   93001 non-null object
nppes_provider_gender               96084 non-null object
nppes_entity_code                   99998 non-null object
nppes_provider_street1              99998 non-null object
nppes_provider_street2              40829 non-null object
nppes_provider_city                 99998 non-null object
nppes_provider_zip                  99998 non-null float64
nppes_provider_state                99998 non-null object
nppes_provider_country              99998 non-null object
provider_type                       99998 non-null object
medicare_participation_indicator    99998 non-null object
place_of_service     

In [18]:
provider_df.describe()

,npi,nppes_provider_zip,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
count,9.999900e+04,9.999800e+04,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000
mean,1.003790e+09,4.175105e+08,234.214318,77.389968,126.474339,102.854484,353.467397,78.486047,79.227825
std,3.440060e+06,3.063483e+08,2570.764198,237.275017,621.004162,287.223528,1023.151181,224.167522,221.550788
min,1.000000e+00,6.240000e+02,11.000000,11.000000,11.000000,0.010000,0.010000,0.000000,0.000000
25%,1.003182e+09,1.514521e+08,21.000000,17.000000,20.000000,24.540000,60.000000,19.538137,20.270000
50%,1.003834e+09,3.660414e+08,43.000000,33.000000,41.000000,65.730000,147.000000,47.267753,48.036030
75%,1.003888e+09,6.811441e+08,118.000000,75.000000,106.000000,114.294286,300.000000,85.613822,85.437644
max,1.013008e+09,9.990157e+08,315121.300000,39569.000000,115002.000000,30493.330000,63025.267273,23906.770000,17413.970000


In [19]:
convicted_df.describe()

,NPI,DOB,ZIP,EXCLDATE,REINDATE,WAIVERDATE
count,7.267600e+04,6.869100e+04,72676.000000,7.267600e+04,72676.0,7.267600e+04
mean,1.096020e+08,1.961971e+07,51420.786642,2.006805e+07,0.0,4.148421e+03
std,3.979338e+08,1.404949e+05,29593.928817,8.143594e+04,0.0,2.887307e+05
min,0.000000e+00,1.901010e+07,0.000000,1.977070e+07,0.0,0.000000e+00
25%,0.000000e+00,1.953032e+07,29918.000000,2.001052e+07,0.0,0.000000e+00
50%,0.000000e+00,1.962041e+07,45694.000000,2.007102e+07,0.0,0.000000e+00
75%,0.000000e+00,1.972050e+07,78416.000000,2.014052e+07,0.0,0.000000e+00
max,1.992979e+09,1.999043e+07,99927.000000,2.019122e+07,0.0,2.017062e+07


Looking for null values. 

In [21]:
pd.isnull(provider_df).any()

npi                                 False
nppes_provider_last_org_name        False
nppes_provider_first_name            True
nppes_provider_mi                    True
nppes_credentials                    True
nppes_provider_gender                True
nppes_entity_code                    True
nppes_provider_street1               True
nppes_provider_street2               True
nppes_provider_city                  True
nppes_provider_zip                   True
nppes_provider_state                 True
nppes_provider_country               True
provider_type                        True
medicare_participation_indicator     True
place_of_service                     True
hcpcs_code                           True
hcpcs_description                    True
hcpcs_drug_indicator                 True
line_srvc_cnt                        True
bene_unique_cnt                      True
bene_day_srvc_cnt                    True
average_Medicare_allowed_amt         True
average_submitted_chrg_amt        

In [22]:
npi_agg = provider_df.groupby('npi') ### grouping the data on npi

In [26]:
npi_agg['npi'].nunique()

npi
1             1
1003000126    1
1003000134    1
1003000142    1
1003000407    1
             ..
1013007723    1
1013007731    1
1013007780    1
1013007798    1
1013007814    1
Name: npi, Length: 10572, dtype: int64

In [28]:
npi_tot = npi_agg['npi'].value_counts() ### Count the number of instances for each NPI